In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
prototxtPath=os.path.sep.join([r'/content/drive/MyDrive/MaskDataset/mobilenet_v2.model','deploy.prototxt'])
weightsPath=os.path.sep.join([r'/content/drive/MyDrive/MaskDataset/mobilenet_v2.model','res10_300x300_ssd_iter_140000.caffemodel'])


In [ ]:
prototxtPath

'/content/drive/MyDrive/MaskDataset/mobilenet_v2.model/deploy.prototxt'

In [ ]:
weightsPath

'/content/drive/MyDrive/MaskDataset/mobilenet_v2.model/res10_300x300_ssd_iter_140000.caffemodel'

In [ ]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)

error: ignored

In [ ]:
model=load_model(r'/content/drive/MyDrive/MaskDataset/mobilenet_v2.model')

In [ ]:
image=cv2.imread(r'/content/drive/MyDrive/MaskDataset/with_mask/10-with-mask.jpg')

In [ ]:
image

In [ ]:
(h,w)=image.shape[:2]

In [ ]:
(h,w)

(350, 233)

In [ ]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [ ]:
blob

array([[[[ 119.,  119.,  119., ...,   92.,   90.,   87.],
         [ 119.,  119.,  120., ...,   93.,   90.,   88.],
         [ 120.,  120.,  120., ...,   94.,   92.,   89.],
         ...,
         [  99.,  105.,  110., ...,  -70.,  -70.,  -71.],
         [  85.,   92.,  100., ...,  -72.,  -72.,  -71.],
         [ 107.,  114.,  117., ...,  -75.,  -74.,  -72.]],

        [[  47.,   47.,   47., ...,   17.,   15.,   12.],
         [  47.,   47.,   48., ...,   18.,   15.,   13.],
         [  48.,   48.,   49., ...,   19.,   17.,   14.],
         ...,
         [  -1.,    6.,   12., ..., -149., -149., -149.],
         [ -16.,   -8.,    1., ..., -150., -150., -149.],
         [   6.,   13.,   17., ..., -152., -151., -150.]],

        [[  99.,   99.,   99., ...,   63.,   61.,   58.],
         [  99.,   99.,  100., ...,   64.,   61.,   59.],
         [ 100.,  100.,  101., ...,   65.,   63.,   60.],
         ...,
         [  23.,   29.,   35., ..., -100.,  -99.,  -98.],
         [   5.,   13.,   

In [ ]:
blob.shape

(1, 3, 300, 300)

In [ ]:
net.setInput(blob)
detections=net.forward()

NameError: ignored

In [ ]:
detections

array([[[[0.        , 1.        , 0.9996724 , ..., 0.18173476,
          0.4329286 , 0.4609872 ],
         [0.        , 1.        , 0.8111837 , ..., 0.02572085,
          0.9942617 , 0.34080845],
         [0.        , 1.        , 0.43454096, ..., 0.19513153,
          0.65238404, 0.6432173 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [ ]:

for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
        
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: ignored